# Tracing for Different Types of Runs

### Types of Runs

LangSmith supports many different types of Runs - you can specify what type your Run is in the @traceable decorator. The types of runs are:

- LLM: Invokes an LLM
- Retriever: Retrieves documents from databases or other sources
- Tool: Executes actions with function calls
- Chain: Default type; combines multiple Runs into a larger process
- Prompt: Hydrates a prompt to be used with an LLM
- Parser: Extracts structured data

### Setup

In [13]:
# You can set them inline!
import os
os.environ["OPENAI_API_KEY"] = "OPEN AI API KEY TO BE USED "
os.environ["LANGSMITH_API_KEY"] = "LANGSMITH API KEY TO BE USED"
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [14]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

False

### LLM Runs for Chat Models

LangSmith provides special rendering and processing for LLM traces. In order to make the most of this feature, you must log your LLM traces in a specific format.

For chat-style models, inputs must be a list of messages in OpenAI-compatible format, represented as Python dictionaries or TypeScript object. Each message must contain the key role and content.

The output is accepted in any of the following formats:

- A dictionary/object that contains the key choices with a value that is a list of dictionaries/objects. Each dictionary/object must contain the key message, which maps to a message object with the keys role and content.
- A dictionary/object that contains the key message with a value that is a message object with the keys role and content.
- A tuple/array of two elements, where the first element is the role and the second element is the content.
- A dictionary/object that contains the key role and content.
The input to your function should be named messages.

You can also provide the following metadata fields to help LangSmith identify the model and calculate costs. If using LangChain or OpenAI wrapper, these fields will be automatically populated correctly.
- ls_provider: The provider of the model, eg "openai", "anthropic", etc.
- ls_model_name: The name of the model, eg "gpt-4o-mini", "claude-3-opus-20240307", etc.

In [31]:
from langsmith import traceable

# Use API key (set in environment as OPENAI_API_KEY and LANGSMITH_API_KEY)

inputs = [
  {"role": "system", "content": "You are a knowledgeable assistant."},
  {"role": "user", "content": "Can you help me book a table for two people?"},
]

output = {
  "choices": [
      {
          "message": {
              "role": "assistant",
              "content": "Absolutely! For what time should I book the table?"
          }
      }
  ]
}

@traceable(
  run_type="llm",
  metadata={"ls_provider": "custom_provider", "ls_model_name": "custom-model-v1"}
)
def chat_model(messages: list):
  print(f"Chat model received messages: {messages}")  # Added debug log
  return output

result = chat_model(inputs)
print("Chat model response:", result)


Chat model received messages: [{'role': 'system', 'content': 'You are a knowledgeable assistant.'}, {'role': 'user', 'content': 'Can you help me book a table for two people?'}]
Chat model response: {'choices': [{'message': {'role': 'assistant', 'content': 'Absolutely! For what time should I book the table?'}}]}


### Handling Streaming LLM Runs

For streaming, you can "reduce" the outputs into the same format as the non-streaming version. This is currently only supported in Python.

In [35]:
from langsmith import traceable

# Use API key (set in environment as OPENAI_API_KEY and LANGSMITH_API_KEY)

def _reduce_chunks(chunks: list):
    concatenated = "".join([chunk["choices"][0]["message"]["content"] for chunk in chunks])
    return {"choices": [{"message": {"content": concatenated, "role": "assistant"}}]}

@traceable(
    run_type="llm",
    metadata={"ls_provider": "custom_provider", "ls_model_name": "custom-model-v1"},
    reduce_fn=_reduce_chunks
)
def streaming_chat(messages: list):
    greeting = "Hello, " + messages[1].get("content", "")
    for partial in [greeting, " How can I assist you today?"]:
        print(f"Streaming chunk: {partial}")  # Debug print
        yield {
            "choices": [
                {
                    "message": {
                        "content": partial,
                        "role": "assistant",
                    }
                }
            ]
        }

collected_chunks = list(
    streaming_chat(
        [
            {"role": "system", "content": "You act as an attentive assistant."},
            {"role": "user", "content": "polly the parrot"},
        ]
    )
)
print("Concatenated streaming response:", collected_chunks)


Streaming chunk: Hello, polly the parrot
Streaming chunk:  How can I assist you today?
Concatenated streaming response: [{'choices': [{'message': {'content': 'Hello, polly the parrot', 'role': 'assistant'}}]}, {'choices': [{'message': {'content': ' How can I assist you today?', 'role': 'assistant'}}]}]


### Retriever Runs + Documents

Many LLM applications require looking up documents from vector databases, knowledge graphs, or other types of indexes. Retriever traces are a way to log the documents that are retrieved by the retriever. LangSmith provides special rendering for retrieval steps in traces to make it easier to understand and diagnose retrieval issues. In order for retrieval steps to be rendered correctly, a few small steps need to be taken.

1. Annotate the retriever step with run_type="retriever".
2. Return a list of Python dictionaries or TypeScript objects from the retriever step. Each dictionary should contain the following keys:
    - page_content: The text of the document.
    - type: This should always be "Document".
    - metadata: A python dictionary or TypeScript object containing metadata about the document. This metadata will be displayed in the trace.

In [36]:
from langsmith import traceable

# Use API key (set in environment as OPENAI_API_KEY and LANGSMITH_API_KEY)

def format_documents(results):
    return [
        {
            "page_content": result,
            "type": "Document",
            "metadata": {"source": f"Source for '{result[:10]}...'"}
        }
        for result in results
    ]

@traceable(run_type="retriever")
def search_documents(query: str):
    sample_docs = [
        "Doc 1: Insights on LangSmith tracing.",
        "Doc 2: Introduction to Python decorators.",
        "Doc 3: Handling API keys securely."
    ]
    print(f"Retriever called with query: {query}")  # Debug log
    return format_documents(sample_docs)

retrieved_docs = search_documents("Tracing basics explanation")
print("Retrieved documents:", retrieved_docs)


Retriever called with query: Tracing basics explanation
Retrieved documents: [{'page_content': 'Doc 1: Insights on LangSmith tracing.', 'type': 'Document', 'metadata': {'source': "Source for 'Doc 1: Ins...'"}}, {'page_content': 'Doc 2: Introduction to Python decorators.', 'type': 'Document', 'metadata': {'source': "Source for 'Doc 2: Int...'"}}, {'page_content': 'Doc 3: Handling API keys securely.', 'type': 'Document', 'metadata': {'source': "Source for 'Doc 3: Han...'"}}]


### Tool Calling

LangSmith has custom rendering for Tool Calls made by the model to make it clear when provided tools are being used.

In [28]:
# Before running this script, set your API keys as environment variables in your terminal or shell:
# export OPENAI_API_KEY="your_openai_api_key_here"
# export LANGSMITH_API_KEY="your_langsmith_api_key_here"

from langsmith import traceable
from openai import OpenAI
from typing import List, Optional
import json
import os

# Alternatively, you can set environment variables here directly (not recommended for production)
# os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"
# os.environ["LANGSMITH_API_KEY"] = "your_langsmith_api_key_here"

openai_client = OpenAI()

@traceable(run_type="tool")
def get_current_temperature(location: str, unit: str):
    temperature = 65 if unit == "Fahrenheit" else 17
    # Changed output string format
    return f"The current temperature in {location} is {temperature} degrees {unit}."

@traceable(run_type="llm")
def call_openai(messages: List[dict], tools: Optional[List[dict]]) -> str:
    return openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0,
        tools=tools
    )

@traceable(run_type="chain")
def ask_about_the_weather(inputs, tools):
    response = call_openai(inputs, tools)
    tool_call_args = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
    location = tool_call_args["location"]
    unit = tool_call_args["unit"]

    tool_response_message = {
        "role": "tool",
        "content": json.dumps({
            "message": get_current_temperature(location, unit),
            "location": location,
            "unit": unit
        }),
        "tool_call_id": response.choices[0].message.tool_calls[0].id
    }

    inputs.append(response.choices[0].message)
    inputs.append(tool_response_message)
    output = call_openai(inputs, None)
    return output

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_temperature",
            "description": "Get the current temperature for a specific location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g., San Francisco, CA"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["Celsius", "Fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the user's location."
                    }
                },
                "required": ["location", "unit"]
            }
        }
    }
]

inputs = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the weather today in New York City?"},
]

result = ask_about_the_weather(inputs, tools)
print(result)


ChatCompletion(id='chatcmpl-CMxrlMebXgMvbH4nF4MINcpPLyBTe', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current temperature in New York City is 65 degrees Fahrenheit. If you need more detailed weather information, feel free to ask!', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1759589377, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_51db84afab', usage=CompletionUsage(completion_tokens=26, prompt_tokens=96, total_tokens=122, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
